In [1]:
import networkx as nx
import json
import ipywidgets as widgets
from IPython.display import display

# List of GraphML files to be combined
graphml_files = '/home/angel/universe/graphrag/mutitxt/output/20240722-155747/artifacts/merged_graph.graphml',
    


# Initialize an empty graph
combined_graph = nx.Graph()

# Load and combine each GraphML file
for graphml_file in graphml_files:
    graph = nx.read_graphml(graphml_file)
    combined_graph = nx.compose(combined_graph, graph)

# Define a color scheme for different types
color_map = {
    'ORGANIZATION': '#008B8B',
    'EVENT': '#8FBC8F',
    'PERSON': '#E9967A',
    'GEO': '#A9A9A9',
    # Add more types and colors as needed
}

# Extract nodes and edges, cleaning up the escape characters
nodes = [{'id': str(node_id).strip('"'), 'type': data.get('type', '').strip('"')} for node_id, data in combined_graph.nodes(data=True)]
edges = [{'source': str(edge[0]).strip('"'), 'target': str(edge[1]).strip('"')} for edge in combined_graph.edges()]

# Convert to JSON format required by 3d-force-graph
graph_data = {
    'nodes': nodes,
    'links': edges
}

# Convert the graph data to JSON string
graph_data_json = json.dumps(graph_data, ensure_ascii=False)

# Create the HTML content for 3d-force-graph
html_content = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <style> body {{ margin: 0; }} </style>
    <script src="https://unpkg.com/3d-force-graph"></script>
</head>
<body>
    <div id="3d-graph" style="width: 100vw; height: 100vh;"></div>

    <script type="importmap"> 
    {{
        "imports": {{ 
            "three": "https://unpkg.com/three/build/three.module.js" 
        }}
    }}
    </script>
    <script type="module">
        import SpriteText from "https://unpkg.com/three-spritetext/dist/three-spritetext.mjs";
        import * as d3 from "https://unpkg.com/d3@6?module";

        const graphData = {graph_data_json};

        const colorMap = {{
            'ORGANIZATION': '#008B8B',
            'EVENT': '#8FBC8F',
            'PERSON': '#E9967A',
            'GEO': '#A9A9A9',
            '': 'gray'  // Default color for undefined types
        }};

        const Graph = ForceGraph3D()
            (document.getElementById('3d-graph'))
                .graphData(graphData)
                .nodeAutoColorBy('type')
                .nodeThreeObject(node => {{
                    const sprite = new SpriteText(node.id);
                    sprite.material.depthWrite = false; // make sprite background transparent
                    sprite.color = colorMap[node.type] || 'gray';
                    sprite.textHeight = 8;
                    return sprite;
                }});

        // Increase repulsion force and link distance
        Graph.d3Force('charge', d3.forceManyBody().strength(-300));
        Graph.d3Force('link', d3.forceLink().distance(200));
    </script>
</body>
</html>
"""

# Display the HTML content using ipywidgets
html_widget = widgets.HTML(value=html_content)



In [2]:
# Save the HTML content to a file
with open('combined_nodes.html', 'w') as f:
    f.write(html_content)
